In [14]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2

# Define directories
data_dir = './videos/MRI515_T1'
label_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

# Directories for segmentation masks
prompt_dir = 'manual_frame_0_402'
prompt_dir = 'manual_prompt_frame0_402'
prompt_dir = 'manual_frame_0_1'

auto_seg_dir1 = os.path.join(data_dir, prompt_dir , 'SAM2_seg_mask_nolap')
auto_seg_dir2 = os.path.join(data_dir, prompt_dir , 'SAM2_seg_mask_yeslap')

# Output CSV path
csv_write_path = os.path.join(data_dir, prompt_dir , prompt_dir + '_dsc_overlap.csv')

# Ensure directories exist
if not os.path.exists(label_dir) or not os.path.exists(auto_seg_dir1) or not os.path.exists(auto_seg_dir2):
    raise FileNotFoundError("One or more required directories do not exist!")

# Function to calculate DSC
def calculate_dsc(mask1, mask2):
    intersection = np.sum((mask1 > 0) & (mask2 > 0))
    union = np.sum(mask1 > 0) + np.sum(mask2 > 0)
    return (2.0 * intersection / union) if union > 0 else 1.0

# Function to calculate overlap ratio
def calculate_overlap_ratio(mask1, mask2):
    intersection = np.sum((mask1 > 0) & (mask2 > 0))
    total_area = np.sum(mask1 > 0) + np.sum(mask2 > 0)
    return (intersection / total_area) if total_area > 0 else 0.0

# Initialize results list
results = []

# Process each label file
for label_file in os.listdir(label_dir):
    label_index = int(label_file.split('.')[0])  # Extract numeric part of filename

    # Construct full file paths
    label_path = os.path.join(label_dir, label_file)

    # Paths for segmentation masks in both directories
    auto_seg_path1_class1 = os.path.join(auto_seg_dir1, f"frame_{label_index}_obj_1.png")
    auto_seg_path1_class2 = os.path.join(auto_seg_dir1, f"frame_{label_index}_obj_2.png")

    auto_seg_path2_class1 = os.path.join(auto_seg_dir2, f"frame_{label_index}_obj_1.png")
    auto_seg_path2_class2 = os.path.join(auto_seg_dir2, f"frame_{label_index}_obj_2.png")

    # Check if auto-segmentation files exist
    if not all(os.path.exists(p) for p in [auto_seg_path1_class1, auto_seg_path1_class2, auto_seg_path2_class1, auto_seg_path2_class2]):
        print(f"Auto-segmentation files missing for {label_file}, skipping.")
        continue

    # Load masks
    label_mask = np.array(Image.open(label_path))

    # Load auto-segmentation masks (nolap)
    auto_mask1_class1 = np.array(Image.open(auto_seg_path1_class1))
    auto_mask1_class2 = np.array(Image.open(auto_seg_path1_class2))

    # Load auto-segmentation masks (yeslap)
    auto_mask2_class1 = np.array(Image.open(auto_seg_path2_class1))
    auto_mask2_class2 = np.array(Image.open(auto_seg_path2_class2))

    # Convert masks to binary (thresholding)
    class1_label_mask = (np.abs(label_mask - 50) < 10).astype(np.uint8)
    class2_label_mask = (np.abs(label_mask - 100) < 10).astype(np.uint8)

    class1_auto_mask_nolap = (np.abs(auto_mask1_class1 - 50) < 10).astype(np.uint8)
    class2_auto_mask_nolap = (np.abs(auto_mask1_class2 - 100) < 10).astype(np.uint8)

    class1_auto_mask_yeslap = (np.abs(auto_mask2_class1 - 50) < 10).astype(np.uint8)
    class2_auto_mask_yeslap = (np.abs(auto_mask2_class2 - 100) < 10).astype(np.uint8)

    # Resize auto-segmentation masks to match label mask size
    class1_auto_mask_nolap = cv2.resize(class1_auto_mask_nolap, (class1_label_mask.shape[1], class1_label_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    class2_auto_mask_nolap = cv2.resize(class2_auto_mask_nolap, (class2_label_mask.shape[1], class2_label_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

    class1_auto_mask_yeslap = cv2.resize(class1_auto_mask_yeslap, (class1_label_mask.shape[1], class1_label_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    class2_auto_mask_yeslap = cv2.resize(class2_auto_mask_yeslap, (class2_label_mask.shape[1], class2_label_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Calculate DSC for class 1 and class 2 (using nolap segmentation)
    class1_dsc = calculate_dsc(class1_label_mask, class1_auto_mask_nolap)
    class2_dsc = calculate_dsc(class2_label_mask, class2_auto_mask_nolap)

    # Calculate areas
    class1_label_area = np.sum(class1_label_mask)
    class1_auto_area = np.sum(class1_auto_mask_nolap)

    class2_label_area = np.sum(class2_label_mask)
    class2_auto_area = np.sum(class2_auto_mask_nolap)

    # Calculate overlap ratio (using yeslap segmentation)
    overlap_ratio = calculate_overlap_ratio(class1_auto_mask_yeslap, class2_auto_mask_yeslap)

    # Store results
    file_results = {
        "file_name": label_file,
        "class1_dsc": class1_dsc,
        "class1_label_area": class1_label_area,
        "class1_auto_area": class1_auto_area,
        "class2_dsc": class2_dsc,
        "class2_label_area": class2_label_area,
        "class2_auto_area": class2_auto_area,
        "overlap_ratio": overlap_ratio
    }

    results.append(file_results)
    print(f"Processed {label_file}: Class1 DSC={class1_dsc}, Class2 DSC={class2_dsc}, Overlap Ratio={overlap_ratio}")

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(csv_write_path, index=False)




Processed 00261.png: Class1 DSC=0.9354920313685808, Class2 DSC=0.9604938271604938, Overlap Ratio=0.0
Processed 00212.png: Class1 DSC=0.9444936521995867, Class2 DSC=0.9316135312728158, Overlap Ratio=0.09630252100840336
Processed 00378.png: Class1 DSC=0.9645944833264718, Class2 DSC=0.9375394321766561, Overlap Ratio=0.013164431197218083
Processed 00071.png: Class1 DSC=0.9218436873747495, Class2 DSC=0.9137891607592042, Overlap Ratio=0.2421727100626183
Processed 00328.png: Class1 DSC=0.9019325064897606, Class2 DSC=0.9408740359897172, Overlap Ratio=0.025292819555253778
Processed 00193.png: Class1 DSC=0.44038499312512275, Class2 DSC=0.6835832372164552, Overlap Ratio=0.3646788990825688
Processed 00401.png: Class1 DSC=0.8774937655860349, Class2 DSC=0.819206008583691, Overlap Ratio=0.2900028320589068
Processed 00049.png: Class1 DSC=0.9525144559313125, Class2 DSC=0.9000786782061369, Overlap Ratio=0.012729549248747913
Processed 00074.png: Class1 DSC=0.9324121357765095, Class2 DSC=0.919287978541819

Processed 00125.png: Class1 DSC=0.8875281179705073, Class2 DSC=0.932152117545376, Overlap Ratio=0.0
Processed 00156.png: Class1 DSC=0.9699235344095157, Class2 DSC=0.9680962343096234, Overlap Ratio=0.0006208609271523179
Processed 00288.png: Class1 DSC=0.9444032616753151, Class2 DSC=0.9251361161524501, Overlap Ratio=0.0008456659619450317
Processed 00323.png: Class1 DSC=0.8690951821386604, Class2 DSC=0.9474683544303798, Overlap Ratio=0.009042102288782142
Processed 00069.png: Class1 DSC=0.9298620555663494, Class2 DSC=0.9473103448275862, Overlap Ratio=0.022328548644338118
Processed 00013.png: Class1 DSC=0.9565950273914876, Class2 DSC=0.9390615478366849, Overlap Ratio=0.02755620014503263
Processed 00358.png: Class1 DSC=0.9651360544217688, Class2 DSC=0.9694774961200207, Overlap Ratio=0.0026004728132387705
Processed 00042.png: Class1 DSC=0.9490354160668011, Class2 DSC=0.969134470105763, Overlap Ratio=0.0005262234695667427
Processed 00411.png: Class1 DSC=0.8650568181818182, Class2 DSC=0.8644929

Processed 00160.png: Class1 DSC=0.9435017713966064, Class2 DSC=0.9571778140293637, Overlap Ratio=0.005982246236974141
Processed 00360.png: Class1 DSC=0.928003104987386, Class2 DSC=0.9623477297895903, Overlap Ratio=0.005397793945083313
Processed 00347.png: Class1 DSC=0.9529532967032966, Class2 DSC=0.9384787472035794, Overlap Ratio=0.00022011886418666079
Processed 00409.png: Class1 DSC=0.936088623775032, Class2 DSC=0.9312052834342323, Overlap Ratio=0.29683992402003107
Processed 00185.png: Class1 DSC=0.931586978636826, Class2 DSC=0.905436447166922, Overlap Ratio=0.11366451080225677
Processed 00223.png: Class1 DSC=0.9450322580645162, Class2 DSC=0.9085014409221902, Overlap Ratio=0.0032535885167464117
Processed 00086.png: Class1 DSC=0.9444444444444444, Class2 DSC=0.95068359375, Overlap Ratio=0.0276962582538518
Processed 00201.png: Class1 DSC=0.8405005688282139, Class2 DSC=0.7603648424543947, Overlap Ratio=0.2678416013925152
Processed 00344.png: Class1 DSC=0.906267332224071, Class2 DSC=0.9317

Processed 00032.png: Class1 DSC=0.9439009782391695, Class2 DSC=0.9331698344573881, Overlap Ratio=0.001250260470931444
Processed 00015.png: Class1 DSC=0.8202579258463192, Class2 DSC=0.9605063291139241, Overlap Ratio=0.01878238341968912
Processed 00330.png: Class1 DSC=0.9500393391030685, Class2 DSC=0.906538856555974, Overlap Ratio=0.023832221163012392
Processed 00213.png: Class1 DSC=0.9361151544557658, Class2 DSC=0.9046119651787368, Overlap Ratio=0.004812834224598931
Processed 00399.png: Class1 DSC=0.9333495204564769, Class2 DSC=0.8984638441363807, Overlap Ratio=0.29725040042712225
Processed 00400.png: Class1 DSC=0.8971887550200803, Class2 DSC=0.9386966450026348, Overlap Ratio=0.0
Processed 00026.png: Class1 DSC=0.9628064243448858, Class2 DSC=0.9356575237118687, Overlap Ratio=0.0005668934240362812
Processed 00075.png: Class1 DSC=0.947817432105787, Class2 DSC=0.9303118908382066, Overlap Ratio=0.11227617357638328
Processed 00006.png: Class1 DSC=0.918092059518843, Class2 DSC=0.8914694059848

In [15]:
results_df.shape

(415, 8)

In [16]:
import pandas as pd

# 假设 results_df 已经被定义
# 定义需要移除的 frame indices，并格式化为 5 位数字
frame_idx = [0, 1, 2]
frame_idx = [0, 1]

frame_idx_patterns = [f"{str(idx).zfill(5)}" for idx in frame_idx]  # ['frame00000', 'frame00001', 'frame00002']

# 构造正则表达式（匹配任何一个 frameXXXXX）
regex_pattern = "|".join(frame_idx_patterns)

# 过滤掉包含这些 frameXXXXX 的行
filtered_results_df = results_df[~results_df['file_name'].str.contains(regex_pattern, regex=True)].copy()

# 显示过滤后的 DataFrame 形状
print("Filtered DataFrame shape:", filtered_results_df.shape)

# 预览前几行数据
print(filtered_results_df.head())


Filtered DataFrame shape: (413, 8)
   file_name  class1_dsc  class1_label_area  class1_auto_area  class2_dsc  \
0  00261.png    0.935492               4161              3745    0.960494   
1  00212.png    0.944494               3457              3317    0.931614   
2  00378.png    0.964594               2429              2429    0.937539   
3  00071.png    0.921844               2840              3148    0.913789   
4  00328.png    0.901933               3776              3158    0.940874   

   class2_label_area  class2_auto_area  overlap_ratio  
0               2393              2467       0.000000  
1               2049              2060       0.096303  
2               1626              1544       0.013164  
3               2317              2056       0.242173  
4               2473              2584       0.025293  


In [17]:
dsc_values_class1,  dsc_values_class2 = [], []
# 遍历 DataFrame
for index, row in filtered_results_df.iterrows():
  
    dsc_values_class1.append(row['class1_dsc'])
    dsc_values_class2.append(row['class2_dsc'])

# 计算并打印 max, mean, min
dsc_values = dsc_values_class1
if dsc_values:
    max_value = max(dsc_values)
    mean_value = sum(dsc_values) / len(dsc_values)
    min_value = min(dsc_values)
    print(len(dsc_values ))

    print("\nStatistics:")

    print(f"Mean: {mean_value:.6f}")
    print('np.std',np.std(dsc_values))
    
dsc_values = dsc_values_class2
if dsc_values:
    max_value = max(dsc_values)
    mean_value = sum(dsc_values) / len(dsc_values)
    min_value = min(dsc_values)
    print("\nStatistics:")
    print(f"Max: {max_value:.6f}", f"Min: {min_value:.6f}")
    print(f"Mean: {mean_value:.6f}")
    print('np.std',np.std(dsc_values))


413

Statistics:
Mean: 0.914058
np.std 0.07035620022342677

Statistics:
Max: 0.980500 Min: 0.386993
Mean: 0.917989
np.std 0.0681183063314966


In [18]:
from scipy.stats import pearsonr, ttest_ind, ks_2samp, shapiro
import numpy as np

results_df = filtered_results_df
# 2. 正态性检验
label_areas = np.array(results_df['class1_label_area'])
auto_seg_areas = np.array(results_df['class1_auto_area'])
label_areas = np.array(results_df['class2_label_area'])
auto_seg_areas = np.array(results_df['class2_auto_area'])

shapiro_label = shapiro(label_areas)
shapiro_auto_seg = shapiro(auto_seg_areas)
print("\nNormality Test Results (Shapiro-Wilk):")
print(f"Label Areas: W={shapiro_label.statistic:.4f}, p={shapiro_label.pvalue:.4e}")
print(f"Auto-Seg Areas: W={shapiro_auto_seg.statistic:.4f}, p={shapiro_auto_seg.pvalue:.4e}")

# 检查正态性结果
label_is_normal = shapiro_label.pvalue > 0.05
auto_seg_is_normal = shapiro_auto_seg.pvalue > 0.05

if label_is_normal and auto_seg_is_normal:
    # 如果两组数据均为正态分布，使用独立两样本 t 检验
    t_stat, p_value_ttest = ttest_ind(label_areas, auto_seg_areas, equal_var=False)
    print("Method: Independent Two-Sample T-Test")
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value_ttest:.4e}")
    print("Interpretation:")
    if p_value_ttest < 0.05:
        print("  - Significant difference detected between the two distributions (p < 0.05).")
    else:
        print("  - No significant difference detected between the two distributions (p >= 0.05).")
else:
    # 如果至少一组数据不是正态分布，使用 Kolmogorov-Smirnov 检验
    ks_stat, p_value_ks = ks_2samp(label_areas, auto_seg_areas)
    print("Method: Two-Sided Kolmogorov-Smirnov Test")
    print(f"KS-statistic: {ks_stat:.4f}")
    print(f"P-value: {p_value_ks:.4f}")
    print("Interpretation:")
    if p_value_ks < 0.05:
        print("  - Significant difference detected between the two distributions (p < 0.05).")
    else:
        print("  - No significant difference detected between the two distributions (p >= 0.05).")


Normality Test Results (Shapiro-Wilk):
Label Areas: W=0.9845, p=2.0740e-04
Auto-Seg Areas: W=0.9900, p=6.4676e-03
Method: Two-Sided Kolmogorov-Smirnov Test
KS-statistic: 0.0678
P-value: 0.2989
Interpretation:
  - No significant difference detected between the two distributions (p >= 0.05).
